In [1]:
import pandas as pd
import numpy as np

In [8]:
# 读取安全的数据 当我们的笔记本内存不够的时候可以采用分块的方式来读取数据
def data_file(filename):
    result = []
    chunk_index = 0
    for df in pd.read_csv(open(filename,'r'),chunksize=1000000):
        result.append(df)
        print('chunk:',chunk_index)
        chunk_index += 1
    result = pd.concat(result,ignore_index=True, axis=0)
    return result
        

In [9]:
train = data_file("./eyi/security_train.csv")
train

chunk: 0
chunk: 1
chunk: 2
chunk: 3
chunk: 4
chunk: 5
chunk: 6
chunk: 7
chunk: 8
chunk: 9
chunk: 10
chunk: 11
chunk: 12
chunk: 13
chunk: 14
chunk: 15
chunk: 16
chunk: 17
chunk: 18
chunk: 19
chunk: 20
chunk: 21
chunk: 22
chunk: 23
chunk: 24
chunk: 25
chunk: 26
chunk: 27
chunk: 28
chunk: 29
chunk: 30
chunk: 31
chunk: 32
chunk: 33
chunk: 34
chunk: 35
chunk: 36
chunk: 37
chunk: 38
chunk: 39
chunk: 40
chunk: 41
chunk: 42
chunk: 43
chunk: 44
chunk: 45
chunk: 46
chunk: 47
chunk: 48
chunk: 49
chunk: 50
chunk: 51
chunk: 52
chunk: 53
chunk: 54
chunk: 55
chunk: 56
chunk: 57
chunk: 58
chunk: 59
chunk: 60
chunk: 61
chunk: 62
chunk: 63
chunk: 64
chunk: 65
chunk: 66
chunk: 67
chunk: 68
chunk: 69
chunk: 70
chunk: 71
chunk: 72
chunk: 73
chunk: 74
chunk: 75
chunk: 76
chunk: 77
chunk: 78
chunk: 79
chunk: 80
chunk: 81
chunk: 82
chunk: 83
chunk: 84
chunk: 85
chunk: 86
chunk: 87
chunk: 88
chunk: 89


,file_id,label,api,tid,index
0,1,5,LdrLoadDll,2488,0
1,1,5,LdrGetProcedureAddress,2488,1
2,1,5,LdrGetProcedureAddress,2488,2
3,1,5,LdrGetProcedureAddress,2488,3
4,1,5,LdrGetProcedureAddress,2488,4
...,...,...,...,...,...
89806688,13887,2,NtClose,2336,618
89806689,13887,2,NtClose,2336,619
89806690,13887,2,NtClose,2336,620
89806691,13887,2,NtClose,2336,621


In [5]:
import os,psutil
mem = psutil.virtual_memory()
print("已经使用的内存：", mem.used/1024/1024)
print("总的内存", mem.total/1024/1024)


已经使用的内存： 14980.75
总的内存 32768.0


In [10]:
test = data_file("./eyi/security_test.csv")

chunk: 0
chunk: 1
chunk: 2
chunk: 3
chunk: 4
chunk: 5
chunk: 6
chunk: 7
chunk: 8
chunk: 9
chunk: 10
chunk: 11
chunk: 12
chunk: 13
chunk: 14
chunk: 15
chunk: 16
chunk: 17
chunk: 18
chunk: 19
chunk: 20
chunk: 21
chunk: 22
chunk: 23
chunk: 24
chunk: 25
chunk: 26
chunk: 27
chunk: 28
chunk: 29
chunk: 30
chunk: 31
chunk: 32
chunk: 33
chunk: 34
chunk: 35
chunk: 36
chunk: 37
chunk: 38
chunk: 39
chunk: 40
chunk: 41
chunk: 42
chunk: 43
chunk: 44
chunk: 45
chunk: 46
chunk: 47
chunk: 48
chunk: 49
chunk: 50
chunk: 51
chunk: 52
chunk: 53
chunk: 54
chunk: 55
chunk: 56
chunk: 57
chunk: 58
chunk: 59
chunk: 60
chunk: 61
chunk: 62
chunk: 63
chunk: 64
chunk: 65
chunk: 66
chunk: 67
chunk: 68
chunk: 69
chunk: 70
chunk: 71
chunk: 72
chunk: 73
chunk: 74
chunk: 75
chunk: 76
chunk: 77
chunk: 78
chunk: 79


In [3]:
%%time
import pickle
with open("./eyi/train.pkl","rb") as f:
    train = pickle.load(f)

with open("./eyi/test.pkl","rb") as f:
    test = pickle.load(f)   
    

CPU times: user 2.25 s, sys: 1.67 s, total: 3.92 s
Wall time: 4.14 s


In [4]:
train.shape,test.shape

# 压缩字段的数据类型大小，降低存储
train['tid'] = train['tid'].astype(np.int8)
train['index'] = train['index'].astype(np.int8)

test['tid'] = test['tid'].astype(np.int8)
test['index'] = test['index'].astype(np.int8)

In [9]:
# 写一个函数来提取相同filed_id下面的不同api调用信息拼接为一个大的字符串
def get_apis(df):
    temp = df.groupby('file_id')
    file_id_apis = {}
    for file_id,grouped in temp:
        res = grouped.sort_values(['tid','index'], ascending=True)
        file_id_apis[file_id] = ' '.join(res['api'])
#         print(file_id_apis)
    return file_id_apis    

train_apis = get_apis(train)   
test_apis = get_apis(test)

In [40]:
# 把baseline里面处理的特征工程增加尽进来
with open('./df_train.pkl','rb') as f:
    df_train = pickle.load(f)
with open('./df_test.pkl','rb') as fi:
    df_test = pickle.load(fi)    

In [41]:
df_train.drop(['tid','api','index'],axis=1, inplace=True)
df_test.drop(['tid','api','index'],axis=1, inplace=True)

In [42]:
# 将 train和test里面的api进行拼接
tte1 = pd.DataFrame.from_dict(train_apis,orient='index',columns=['api'])
tte1 = tte1.reset_index().rename(columns={'index':'file_id'})

In [43]:
# 将api的特征和baseline的特征数据df进行Merge
df_train = df_train.merge(tte1, on='file_id',how='left' )
df_train


,file_id,label,api_count,api_min,api_max,api_mean,api_std,api_median,api_ptp,api_nunique,...,index_count,index_min,index_max,index_mean,index_std,index_median,index_ptp,index_nunique,index_skew,api
0,1,5,6786,-123,124,-37.972885,68.022112,-30.0,-9,112,...,6786,-128,127,2.377247,73.888939,6.0,-1,256,-0.070184,LdrLoadDll CoInitializeEx NtDeviceIoControlFil...
1,2,2,816,-123,97,-74.970588,59.256408,-100.0,-36,30,...,816,-128,127,6.127451,81.164974,25.5,-1,204,-0.229294,NtMapViewOfSection NtClose NtClose NtCreateFil...
2,3,0,463,-123,94,-63.406048,62.612145,-83.0,-39,42,...,463,-128,127,2.092873,77.205452,12.0,-1,256,-0.094421,LdrGetProcedureAddress RegOpenKeyExW LdrGetPro...
3,4,0,2046,-124,108,-47.382698,77.081310,-77.0,-24,51,...,2046,-128,127,-0.487292,73.954288,0.0,-1,256,-0.000516,NtAllocateVirtualMemory NtDuplicateObject NtCl...
4,5,0,10002,-123,108,-41.104379,41.440128,-25.0,-25,65,...,10002,-128,127,0.915817,74.009630,2.0,-1,256,-0.032673,LdrLoadDll RegQueryValueExW RegQueryValueExW R...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13882,13883,2,178221,-123,115,-50.109280,63.853510,-59.0,-18,71,...,178221,-128,127,28.664226,44.776345,28.0,-1,256,-1.037521,GetSystemTimeAsFileTime GetSystemTimeAsFileTim...
13883,13884,5,1319,-123,115,-8.159212,81.807449,23.0,-18,39,...,1319,-128,127,0.076573,72.927778,3.0,-1,256,-0.022622,NtDuplicateObject LoadStringW LoadStringW Load...
13884,13885,0,1033,-123,108,-57.808325,56.008876,-67.0,-25,71,...,1033,-128,127,-0.460794,73.614862,0.0,-1,256,-0.001597,FindResourceExW NtClose NtClose NtDuplicateObj...
13885,13886,1,5316,-124,108,-46.079759,65.422040,-55.0,-24,79,...,5316,-128,127,2.847254,73.186256,9.0,-1,256,-0.109383,NtAllocateVirtualMemory LdrGetDllHandle LdrGet...


In [44]:
test_1 = pd.DataFrame.from_dict(test_apis,orient='index',columns=['api'])
test_1 = test_1.reset_index().rename(columns={'index':'file_id'})
df_test = df_test.merge(test_1, on='file_id',how='left' )
df_test

,file_id,api_count,api_min,api_max,api_mean,api_std,api_median,api_ptp,api_nunique,api_skew,...,index_count,index_min,index_max,index_mean,index_std,index_median,index_ptp,index_nunique,index_skew,api
0,1,97,-123,23,-78.896907,40.710739,-99.0,-110,15,1.324342,...,97,0,30,14.443299,9.210466,14.0,30,31,0.048650,RegOpenKeyExA CopyFileA OpenSCManagerA CreateS...
1,2,1361,-123,98,-69.445261,71.970726,-118.0,-35,40,1.166088,...,1361,-128,127,5.125643,76.209674,14.0,-1,256,-0.173416,FindResourceExW LoadResource NtClose LoadResou...
2,3,16,-122,95,-32.625000,82.383554,-3.0,-39,9,0.093997,...,16,0,15,7.500000,4.760952,7.5,15,16,0.000000,GetSystemTimeAsFileTime SetUnhandledExceptionF...
3,4,193,-123,97,-71.844560,42.848409,-59.0,-36,34,0.771970,...,193,-128,127,9.782383,82.105460,31.0,-1,193,-0.344189,StartServiceA NtOpenKeyEx RegSetValueExA NtCre...
4,5,803,-123,97,-43.514321,68.629300,-42.0,-36,34,0.723286,...,803,-128,127,-0.245330,72.330123,2.0,-1,256,-0.010543,RegCreateKeyExW RegCloseKey RegQueryValueExW R...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12950,12951,289,-123,116,-55.228374,69.093157,-86.0,-17,37,0.805345,...,289,-128,127,42.519031,68.138904,55.0,-1,145,-1.307345,NtDuplicateObject NtClose NtClose NtDuplicateO...
12951,12952,112,-123,108,-64.901786,62.533589,-92.0,-25,28,1.454941,...,112,0,111,55.500000,32.475632,55.5,111,112,0.000000,NtAllocateVirtualMemory NtFreeVirtualMemory Nt...
12952,12953,5095,-128,127,-50.510304,62.066683,-29.0,-1,69,-0.036750,...,5095,-128,127,3.662610,73.730197,9.0,-1,256,-0.111519,NtDuplicateObject RegOpenKeyExW FindFirstFileE...
12953,12954,2951,-123,105,-53.280922,68.843867,-77.0,-28,62,0.182890,...,2951,-128,127,5.238563,73.641551,12.0,-1,256,-0.149793,LdrGetProcedureAddress Thread32Next Thread32Ne...


In [45]:
# 将Train 和test的结果进行合并
df_all = pd.concat([df_train, df_test], axis=0)
df_all

,file_id,label,api_count,api_min,api_max,api_mean,api_std,api_median,api_ptp,api_nunique,...,index_count,index_min,index_max,index_mean,index_std,index_median,index_ptp,index_nunique,index_skew,api
0,1,5.0,6786,-123,124,-37.972885,68.022112,-30.0,-9,112,...,6786,-128,127,2.377247,73.888939,6.0,-1,256,-0.070184,LdrLoadDll CoInitializeEx NtDeviceIoControlFil...
1,2,2.0,816,-123,97,-74.970588,59.256408,-100.0,-36,30,...,816,-128,127,6.127451,81.164974,25.5,-1,204,-0.229294,NtMapViewOfSection NtClose NtClose NtCreateFil...
2,3,0.0,463,-123,94,-63.406048,62.612145,-83.0,-39,42,...,463,-128,127,2.092873,77.205452,12.0,-1,256,-0.094421,LdrGetProcedureAddress RegOpenKeyExW LdrGetPro...
3,4,0.0,2046,-124,108,-47.382698,77.081310,-77.0,-24,51,...,2046,-128,127,-0.487292,73.954288,0.0,-1,256,-0.000516,NtAllocateVirtualMemory NtDuplicateObject NtCl...
4,5,0.0,10002,-123,108,-41.104379,41.440128,-25.0,-25,65,...,10002,-128,127,0.915817,74.009630,2.0,-1,256,-0.032673,LdrLoadDll RegQueryValueExW RegQueryValueExW R...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12950,12951,NaN,289,-123,116,-55.228374,69.093157,-86.0,-17,37,...,289,-128,127,42.519031,68.138904,55.0,-1,145,-1.307345,NtDuplicateObject NtClose NtClose NtDuplicateO...
12951,12952,NaN,112,-123,108,-64.901786,62.533589,-92.0,-25,28,...,112,0,111,55.500000,32.475632,55.5,111,112,0.000000,NtAllocateVirtualMemory NtFreeVirtualMemory Nt...
12952,12953,NaN,5095,-128,127,-50.510304,62.066683,-29.0,-1,69,...,5095,-128,127,3.662610,73.730197,9.0,-1,256,-0.111519,NtDuplicateObject RegOpenKeyExW FindFirstFileE...
12953,12954,NaN,2951,-123,105,-53.280922,68.843867,-77.0,-28,62,...,2951,-128,127,5.238563,73.641551,12.0,-1,256,-0.149793,LdrGetProcedureAddress Thread32Next Thread32Ne...


In [46]:
# 来对api字段文本进行tfidf关键字的特征提起
from sklearn.feature_extraction.text import TfidfVectorizer

# 
vec = TfidfVectorizer(ngram_range=(1,3),min_df=0.1)
api_featutres = vec.fit_transform(df_all['api'])


In [47]:
df_apiss = pd.DataFrame(api_featutres.toarray(), columns=vec.get_feature_names_out())

# df_apiss
# 将train 和test的分开，前面我们是放在一起做的
df_train_api = df_apiss[df_apiss.index <= 13886 ]
df_test_api = df_apiss[df_apiss.index > 13886 ]

In [48]:
# 需要对df_test的range处理一下.
df_test_api.index = range(len(df_test))
df_test_api

,__exception__,__exception__ ldrgetprocedureaddress,closesocket,cocreateinstance,cocreateinstance cocreateinstance,cocreateinstance ldrgetprocedureaddress,cocreateinstance ntallocatevirtualmemory,cocreateinstance ntclose,cocreateinstance regclosekey,cocreateinstance regopenkeyexw,...,thread32next thread32next ntprotectvirtualmemory,thread32next thread32next ntquerysysteminformation,thread32next thread32next ntterminateprocess,thread32next thread32next thread32first,thread32next thread32next thread32next,uuidcreate,uuidcreate ntopenthread,uuidcreate ntopenthread ldrunloaddll,writeconsolew,wsastartup
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,0.000000,0.000000,0.000000,0.003446,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12950,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033091,0.000000,0.000000,0.0,0.000000
12951,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
12952,0.000421,0.000584,0.000000,0.001724,0.001174,0.0,0.000332,0.0,0.0,0.0,...,0.003847,0.000648,0.000648,0.000648,0.343158,0.000520,0.000663,0.000671,0.0,0.000000
12953,0.000405,0.000563,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.003706,0.000624,0.000624,0.000624,0.340147,0.000000,0.000000,0.000000,0.0,0.000836


In [49]:
# 将tfidf 的特征和原来的特征合并
df_train = df_train.merge(df_train_api, left_index=True, right_index=True)
df_test = df_train.merge(df_test_api, left_index=True, right_index=True)

In [50]:
# df_all[df_all['api'] == 0]

In [15]:
# train['api'] = df_all['api'].loc[:89806693]

In [56]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13887 entries, 0 to 13886
Columns: 1873 entries, file_id to wsastartup
dtypes: float64(1855), int64(8), int8(9), object(1)
memory usage: 198.2+ MB


In [57]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12955 entries, 0 to 12954
Columns: 3716 entries, file_id to wsastartup_y
dtypes: float64(3698), int64(8), int8(9), object(1)
memory usage: 366.6+ MB


In [27]:
# # 查看某个变量的资源使用
# import sys

# sys.getsizeof(df_all)/1024/1024


4354.063846588135

In [30]:
# 针对不适用的变量进行垃圾回收
import gc

# del df_all
gc.collect()

262719

In [35]:
# 构造特征聚合特征 根据file_id进行聚合

def get_features(df):
    # 按照file_id分组，提取统计值
    df_file = df.groupby('file_id')
    if 'label' in df.columns:
        # 训练集
        df1 = df.drop_duplicates(subset=['file_id', 'label'],keep='first')
    else: # 测试集
        df1 = df.drop_duplicates(subset=['file_id'],keep='first')
    df1 = df1.sort_values('file_id')
    # 提取多个特征的统计特征 api, tid,index
    featues= ['api','tid','index']
    for f in featues:
        df1[f+'_count'] = df_file[f].count().values
        df1[f+'_min'] = df_file[f].min().values
        df1[f+'_max'] = df_file[f].max().values
        df1[f+'_mean'] = df_file[f].mean().values
        df1[f+'_std'] = df_file[f].std().values
        df1[f+'_median'] = df_file[f].median().values
        df1[f+'_ptp'] = df1[f+'_max'] - df1[f+'_min']
        df1[f+'_nunique'] = df_file[f].nunique().values
        df1[f+'_skew'] = df_file[f].skew().values

    return df1    
   

In [36]:
%%time
df_train = get_features(train) 

CPU times: user 1min 24s, sys: 3.77 s, total: 1min 28s
Wall time: 1min 28s


In [48]:
%%time
df_test = get_features(test)

CPU times: user 1min 11s, sys: 3.44 s, total: 1min 14s
Wall time: 1min 15s


1        1
8641     1
8631     1
8632     1
8633     1
        ..
4322     1
4323     1
4324     1
4325     1
12955    1
Name: file_id, Length: 12955, dtype: int64

In [50]:
# 将最后处理好的数据pickle保存
df_train.to_pickle('./df_train.pkl')
df_test.to_pickle('./df_test.pkl')

In [51]:
# 将数据读取尽进来
with open('./df_train.pkl','rb') as f:
    df_train = pickle.load(f)
with open('./df_test.pkl','rb') as f:
    df_test = pickle.load(f)    

In [52]:
df_test['file_id'].value_counts()

1        1
8641     1
8631     1
8632     1
8633     1
        ..
4322     1
4323     1
4324     1
4325     1
12955    1
Name: file_id, Length: 12955, dtype: int64

In [51]:
import lightgbm as gbt

model = gbt.LGBMClassifier(
                            num_leaves=2**5 -1, reg_alpha=0.25, reg_lambda=0.25,objective='multiclass',
                          max_depth=-1,learning_rate=0.005,min_child_samples=3,random_state=2022,n_estimators=3000,
                           subsample=1.0,colsample_bytree=1.0    
)

In [52]:
%%time
model.fit(df_train.drop(['label','api'],axis=1), df_train['label'])

CPU times: user 2h 16min 37s, sys: 12min 56s, total: 2h 29min 33s
Wall time: 15min 51s


LGBMClassifier(learning_rate=0.005, min_child_samples=3, n_estimators=3000,
               objective='multiclass', random_state=2022, reg_alpha=0.25,
               reg_lambda=0.25)

In [55]:
result = model.predict_proba(df_test)

ValueError: Number of features of the model must match the input. Model n_features_ is 1871 and input n_features is 3716

In [43]:
result

array([[2.05228532e-02, 9.92277012e-04, 5.93873812e-01, ...,
        7.66054524e-02, 5.96201154e-02, 2.17415426e-01],
       [5.80096171e-01, 3.82649270e-04, 7.63953116e-03, ...,
        1.63103909e-02, 1.70861028e-01, 2.04590284e-01],
       [9.98592139e-01, 8.81181065e-05, 3.26703086e-04, ...,
        4.65775548e-04, 5.88797782e-05, 4.20642469e-04],
       ...,
       [1.51626314e-03, 6.49609139e-05, 1.16787347e-03, ...,
        9.88598779e-01, 5.29111175e-03, 2.21830321e-03],
       [1.77252478e-03, 5.61618856e-05, 6.89120267e-04, ...,
        9.89166102e-01, 6.75082331e-04, 1.75230882e-03],
       [2.01714457e-03, 9.15267463e-05, 6.54435304e-04, ...,
        9.94493165e-01, 1.37332950e-04, 2.38169115e-03]])

In [44]:
result.shape

(12955, 8)

In [53]:
columns = ['prob0', 'prob1', 'prob2', 'prob3', 'prob4', 'prob5' ,'prob6','prob7']
res = pd.DataFrame(result,columns=columns)
res['file_id'] = df_test['file_id'].values

In [54]:
res

,prob0,prob1,prob2,prob3,prob4,prob5,prob6,prob7,file_id
0,0.020523,0.000992,0.593874,0.030053,0.000917,0.076605,0.059620,0.217415,1
1,0.580096,0.000383,0.007640,0.019871,0.000249,0.016310,0.170861,0.204590,2
2,0.998592,0.000088,0.000327,0.000043,0.000004,0.000466,0.000059,0.000421,3
3,0.392183,0.002601,0.002917,0.032290,0.000891,0.257945,0.022139,0.289033,4
4,0.766479,0.000468,0.162786,0.002427,0.000067,0.049887,0.002697,0.015188,5
...,...,...,...,...,...,...,...,...,...
12950,0.659500,0.003581,0.042596,0.036280,0.000823,0.090585,0.007971,0.158663,12951
12951,0.973950,0.000063,0.000967,0.000272,0.000843,0.011989,0.000157,0.011759,12952
12952,0.001516,0.000065,0.001168,0.001111,0.000032,0.988599,0.005291,0.002218,12953
12953,0.001773,0.000056,0.000689,0.005804,0.000085,0.989166,0.000675,0.001752,12954


In [55]:
# 需要对字段调整一下顺序file_id再前面
new_col = ['file_id','prob0', 'prob1', 'prob2', 'prob3', 'prob4', 'prob5' ,'prob6','prob7']
res.to_csv('./gbm_tfidf.csv',index=False,columns=new_col)

In [56]:
# 读取看一下
pd.read_csv('./gbm_tfidf.csv')

,file_id,prob0,prob1,prob2,prob3,prob4,prob5,prob6,prob7
0,1,0.020523,0.000992,0.593874,0.030053,0.000917,0.076605,0.059620,0.217415
1,2,0.580096,0.000383,0.007640,0.019871,0.000249,0.016310,0.170861,0.204590
2,3,0.998592,0.000088,0.000327,0.000043,0.000004,0.000466,0.000059,0.000421
3,4,0.392183,0.002601,0.002917,0.032290,0.000891,0.257945,0.022139,0.289033
4,5,0.766479,0.000468,0.162786,0.002427,0.000067,0.049887,0.002697,0.015188
...,...,...,...,...,...,...,...,...,...
12950,12951,0.659500,0.003581,0.042596,0.036280,0.000823,0.090585,0.007971,0.158663
12951,12952,0.973950,0.000063,0.000967,0.000272,0.000843,0.011989,0.000157,0.011759
12952,12953,0.001516,0.000065,0.001168,0.001111,0.000032,0.988599,0.005291,0.002218
12953,12954,0.001773,0.000056,0.000689,0.005804,0.000085,0.989166,0.000675,0.001752
